In [1]:
#creating our own dataset
import cv2 #images
import numpy as np #calculations
import os #folders

In [2]:
image_x, image_y = 50, 50 #width and height of img

In [3]:
def create_folder(folder_name): #folder nameis sent as parameter
    if not os.path.exists(folder_name): #checks if the folder name already exists or not. If does not exist, then creates that folder.
        os.mkdir(folder_name)

In [4]:
def store_images(g_id): #get gesture id as parameter
    total_pics = 1200
    cap = cv2.VideoCapture(0) #opening webcam
    x, y, w, h = 300, 50, 350, 350 #particular area where we will capture gesture
    
    create_folder("gestures/" + str(g_id)) #inside gesture folder, creates the folder with gesture id(g_id)
    
    #local vars, whose values change, when we are actually creating images
    pic_no = 0
    flag_start_capturing = False
    frames = 0
    
    #Adding filters to images
    while True:
        ret, frame = cap.read() #Reading the webcam and storing it in frame var
        frame = cv2.flip(frame, 1) #in webcam everything is inverted, so we are flipping the image again, for it to look in the correct format
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) #used to convert image to hsv format
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255])) #The color of the mask2 is betn the 2 given np array values, which is the skin color value.
        res = cv2.bitwise_and(frame, frame, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)
        
        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2) #One pixel on the border s removed, and other pixels in that color is removed to remove background noise
        opening=cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel_square) #smoothen the image more

        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)
        thresh = thresh[y:y + h, x:x + w] #black n white
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1] #looking for different handshakes
        
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 10000 and frames > 50:
                x1, y1, w1, h1 = cv2.boundingRect(contour) #gives min. bounding rectangle
                pic_no += 1
                save_img = thresh[y1:y1 + h1, x1:x1 + w1]
                save_img = cv2.resize(save_img, (image_x, image_y))
                cv2.putText(frame, "Capturing...", (30, 60), cv2.FONT_HERSHEY_TRIPLEX, 2, (127, 255, 255))
                cv2.imwrite("gestures/" + str(g_id) + "/" + str(pic_no) + ".jpg", save_img)
        
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, str(pic_no), (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (127, 127, 255))
        cv2.imshow("Capturing gesture", frame)
        cv2.imshow("thresh", thresh)
        keypress = cv2.waitKey(1)
        if keypress == ord('c'):
            if flag_start_capturing == False:
                flag_start_capturing = True
            else:
                flag_start_capturing = False
                frames = 0
        if flag_start_capturing == True:
            frames += 1
        if pic_no == total_pics:
            break

In [ ]:
g_id = input("Enter gesture number: ")
store_images(g_id)

Enter gesture number: 5
